# 快速入门 语音识别模型 Whisper 

OpenAI 提供了两个基于开源的 Whisper large-v2 模型的语音到文本API服务：
- 转录（transcriptions）：将音频转录为音频所使用的任何语言。
- 翻译（translations）：将音频翻译并转录为英语


目前文件上传限制为 25 MB，支持以下输入文件类型：`mp3、mp4、mpeg、mpga、m4a、wav 和 webm`。


## 语音转录 Transcription API

输入音频文件，返回转录对象（JSON）

参数
- **file**（文件）：需要转录的音频文件对象（不是文件名），支持以下格式：flac、mp3、mp4、mpeg、mpga、m4a、ogg、wav 或 webm。
- **model**（'whisper-1'）：使用的模型 ID。目前仅可使用由我们的开源 Whisper V2 模型驱动的 whisper-1。
- **language**（语言，可选）：输入音频的语言。提供 ISO-639-1 格式的输入语言可以提高准确性和响应速度。
- **prompt**（提示，可选）：可选文本，用于指导模型的风格或继续前一个音频片段。提示应与音频语言相匹配。
- **response_format**（响应格式，可选）：转录输出的格式，默认为 json。可选的格式有：json、text、srt、verbose_json 或 vtt。
- **temperature**（温度，可选）：采样温度，范围从 0 到 1。更高的值，如 0.8，将使输出更随机，而更低的值，如 0.2，将使输出更集中和确定。如果设置为 0，模型将使用对数概率自动提高温度，直到达到某些阈值。
- **timestamp_granularities[]**（时间戳粒度，可选）：为此转录填充的时间戳粒度，默认为 segment。响应格式必须设置为 verbose_json 才能使用时间戳粒度。支持以下一个或两个选项：word 或 segment。注意：segment 时间戳不增加额外延迟，但生成 word 时间戳会增加额外延迟。

返回值
- 转录对象（Transcription Object）或详细转录对象（Verbose Transcription Object）。

### 转录对象

**Transcription Object**:
```json
{
  "text": "Imagine the wildest idea that you've ever had, and you're curious about how it might scale to something that's a 100, a 1,000 times bigger. This is a place where you can get to do that."
}
```

**Verbose Transcription Object**:
```json
{
  "task": "transcribe",
  "language": "english",
  "duration": 8.470000267028809,
  "text": "The beach was a popular spot on a hot summer day. People were swimming in the ocean, building sandcastles, and playing beach volleyball.",
  "segments": [
    {
      "id": 0,
      "seek": 0,
      "start": 0.0,
      "end": 3.319999933242798,
      "text": " The beach was a popular spot on a hot summer day.",
      "tokens": [
        50364, 440, 7534, 390, 257, 3743, 4008, 322, 257, 2368, 4266, 786, 13, 50530
      ],
      "temperature": 0.0,
      "avg_logprob": -0.2860786020755768,
      "compression_ratio": 1.2363636493682861,
      "no_speech_prob": 0.00985979475080967
    },
    ...
  ]
}

```

### 使用 Whisper 实现中文转录

将 TTS 配音的李云龙台词音频文件(liyunlong.mp3)发送给 Whisper 模型进行中文转录

In [1]:
from openai import OpenAI
client = OpenAI(base_url="https://api.xiaoai.plus/v1", api_key="sk-tdqfro61NniG2LZd5394E5F28c51419d8b4dE7Db5a6105C4")

audio_file= open("./audio/liyunlong.mp3", "rb")

transcription = client.audio.transcriptions.create(
  model="whisper-1", 
  file=audio_file
)

print(transcription.text)

二营长,你他娘的意大利泡呢?给我拉来!


## 语音翻译 API

输入音频文件，返回翻译文本

请求体
- **file**（文件）：需要翻译的音频文件对象（不是文件名），支持以下格式：flac、mp3、mp4、mpeg、mpga、m4a、ogg、wav 或 webm。
- **model**（'whisper-1'）：使用的模型 ID。目前只有由我们的开源 Whisper V2 模型驱动的 whisper-1 可用。
- **prompt**（提示，可选）：可选文本，用于指导模型的风格或继续前一个音频片段。提示应为英文。
- **response_format**（响应格式，可选）：转录输出的格式，默认为 json。可选的格式包括：json、text、srt、verbose_json 或 vtt。
- **temperature**（温度，可选）：采样温度，范围从 0 到 1。较高的值，如 0.8，将使输出更随机，而较低的值，如 0.2，将使输出更集中和确定。如果设置为 0，模型将使用对数概率自动增加温度，直到达到特定阈值。

返回值
- **translated_text**: 翻译后的文本。

### 使用 Whisper 实现中文识别+翻译

将 TTS 配音的李云龙台词音频文件(liyunlong.mp3)发送给 Whisper 模型进行翻译

In [6]:
audio_file= open("./audio/liyunlong.mp3", "rb")

translation = client.audio.translations.create(
    model="whisper-1", 
    file=audio_file,
    prompt="Translate into English",
)

print(translation.text)

BadRequestError: Error code: 400 - {'error': {'message': "bind request body failed: Key: 'TextToSpeechRequest.Model' Error:Field validation for 'Model' failed on the 'required' tag\nKey: 'TextToSpeechRequest.Input' Error:Field validation for 'Input' failed on the 'required' tag\nKey: 'TextToSpeechRequest.Voice' Error:Field validation for 'Voice' failed on the 'required' tag (request id: ) (request id: 20240723133104316309372nZWvc3qW)", 'type': 'chat_api_error', 'param': '', 'code': 'bind_request_body_failed'}}

### 使用 TTS 给李云龙英文版台词配音

In [3]:
speech_file_path = "./audio/liyunlong_en.mp3"

with client.audio.speech.with_streaming_response.create(
    model="tts-1",
    voice="onyx",
    input=translation.text
) as response:
    response.stream_to_file(speech_file_path)

NameError: name 'translation' is not defined

### 使用 Whipser + TTS 生成郭德纲相声英文版

In [5]:
gdg_audio_file = open("./audio/gdg.mp3", "rb")
gdg_speech_file = "./audio/gdg_en.mp3"

translation = client.audio.translations.create(
  model="whisper-1", 
  file=gdg_audio_file
)

print(translation.text)

with client.audio.speech.with_streaming_response.create(
    model="tts-1",
    voice="onyx",
    input=translation.text
) as response:
    response.stream_to_file(gdg_speech_file)

BadRequestError: Error code: 400 - {'error': {'message': "bind request body failed: Key: 'TextToSpeechRequest.Model' Error:Field validation for 'Model' failed on the 'required' tag\nKey: 'TextToSpeechRequest.Input' Error:Field validation for 'Input' failed on the 'required' tag\nKey: 'TextToSpeechRequest.Voice' Error:Field validation for 'Voice' failed on the 'required' tag (request id: ) (request id: 20240723115908902052004VgFZduQ0)", 'type': 'chat_api_error', 'param': '', 'code': 'bind_request_body_failed'}}